## Importing packages

In [1]:
import anndata as an
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.widgets import Slider

import scipy.stats as stats
import random
import joblib
from joblib import Parallel, delayed
import multiprocessing
import itertools
import time

import sklearn.gaussian_process as gp
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF, ConstantKernel

from scipy.stats import norm
from scipy.optimize import minimize
import torch
import gpytorch
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split

from gpytorch.constraints.constraints import Interval

import re
import os
import copy

## Import required classes and functions from files in src folder

In [2]:
from src.GP_models import ExactGPModel, TorchGPModel
from src.bayesian_opt_code import ucb, ES, sample_next_hyperparameter

## Load data and update GP model. Also load pseudotimes of devices for which we wish to sample new actions this round.

In [3]:
data = np.load('test_data/data.npz')
xp = data['xp']
yp = data['yp'].reshape(20)
s = data['pseudotime_current_devices']
model = TorchGPModel(torch.tensor(xp).float(), torch.tensor(yp).float())

## Set parameters. Variables are defined as follows.
### bounds: bounds on the state/pseudotime (first dimension) and action space (2-dimensional). 
### batch_size: number of points to sample in parallel in one round
### s_dim: dimension of the pseudotime (1-dim in this problem)
### a_dim: dimension of the action (2-dim in this problem)

In [4]:
bounds = np.asarray([[0,1],[0,1],[0,1]])
batch_size = 3
s_dim = 1
a_dim = 2

## Perform parallel Entropy Search to select next acquisition points 

In [5]:
use_ES = True
next_sa = np.empty((batch_size, s_dim + a_dim))
ES_init_points = 10
yp = np.copy(yp).reshape(20)
for i in range(batch_size):
    next_sa[i,s_dim:] = sample_next_hyperparameter(s[i], model, yp, acquisition_func = ucb,acq = "ucb",greater_is_better=True, 
                                                   bounds=bounds)
    next_sa[i,:s_dim] = s[i]
if use_ES == True:
    a_new = ES( next_sa,
                model,
                state_dim=1,
                n_params=2,
                beta=0.2,
                alpha=0.0,
                a_circ=0.2,
                tr_iters=20,
                domain=bounds[0, :],
                init_points=ES_init_points,
            )
    next_sa[:, s_dim:] = a_new

best a [[0.1037811  0.7469702 ]
 [0.         0.34992424]
 [0.01       0.9528187 ]]


## Print proposed proposal of new actions to sample 

In [9]:
    print(
        f"device joint proposal after using ES: ",
        next_sa[:, s_dim:])
    print("state is ",
        s)

device joint proposal after using ES:  [[0.1037811  0.74697018]
 [0.         0.34992424]
 [0.01       0.95281869]]
state is  [0.3  0.35 0.4 ]
